<a href="https://colab.research.google.com/github/qsardor/GoogleColabProjects/blob/main/FaceFusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **Install Dependencies**
# Clone repository
!git clone -q https://github.com/splendormagic/Bahaa-Ai-DeepFake roop
%cd roop

# Fix dependencies
!sed -i 's/numpy==1.24.3/numpy==1.26.4/g' requirements.txt
!sed -i 's/tensorflow==2.14.0/tensorflow==2.16.1/g' requirements.txt

# Install dependencies
!pip install --quiet --extra-index-url https://download.pytorch.org/whl/cu118 -r requirements.txt --force-reinstall

# Set CUDA environment variables
import os
os.environ["CUDA_PATH"] = "/usr/local/cuda"
os.environ["PATH"] += ":/usr/local/cuda/bin"
os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda/lib64"

In [ ]:
#@title **Run Job (Google Drive Integration)**
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Configure paths
DRIVE_DIR = "/content/drive/MyDrive/Deepfake"
INPUT_DIR = os.path.join(DRIVE_DIR, "input")
OUTPUT_DIR = os.path.join(DRIVE_DIR, "output")
SOURCE_IMG = os.path.join(INPUT_DIR, "source.png")
TARGET_IMG = os.path.join(INPUT_DIR, "target.png")
RESULT_IMG = os.path.join(OUTPUT_DIR, "result.png")

# Create directories
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Validate input files
if not (os.path.exists(SOURCE_IMG) and os.path.exists(TARGET_IMG)):
    raise FileNotFoundError(f"Missing source.png or target.png in {INPUT_DIR}")

# Run processing
%cd /content/roop
!python run.py --source {SOURCE_IMG} --target {TARGET_IMG} --output {RESULT_IMG} --execution-provider cuda --frame-processor face_swapper

print(f"Result saved to: {RESULT_IMG}")

2025-03-23 13:31:03.044893: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742736663.064432    5204 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742736663.070693    5204 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
usage: run.py [-h] [-s SOURCE_PATH] [-t TARGET_PATH] [-o OUTPUT_PATH]
              [--frame-processor FRAME_PROCESSOR [FRAME_PROCESSOR ...]] [--keep-fps]
              [--keep-frames] [--skip-audio] [--many-faces]
              [--reference-face-position REFERENCE_FACE_POSITION]
              [--reference-frame-number REFERENCE_FRAME_NUMBER]
              [--similar-face-distance SIMILAR_FACE_DISTANCE] [--temp-frame-format {jpg,png}

In [ ]:
#@title **Run Job (Local Files)**
import os

# Check local files
if not (os.path.exists("source.png") and os.path.exists("target.png")):
    raise FileNotFoundError("Upload source.png and target.png to Colab")

# Run processing
%cd /content/roop
!python run.py --source source.png --target target.png --output result.png --execution-provider cuda --frame-processor face_swapper

print("Result saved to: /content/roop/result.png")